# Play Cart Pole

In [ ]:
import gym
env = gym.make("CartPole-v0") #https://gym.openai.com/envs/#board_game

In [ ]:
print(env.observation_space.low, env.observation_space.high)
#cart position/velocity + pole angle/velocity at tip

In [ ]:
env.action_space.n  #http://gym.openai.com/docs/

In [ ]:
from gym.utils import play
try: 
    play.play(env)
finally:
    env.close()

In [ ]:
# Play randomly
env.reset()
for _ in range(1000):
    env.render()
    env.step(env.action_space.sample()) # take a random action

env.close()

In [ ]:
env.reset()
new_state, reward, done, info = env.step(env.action_space.sample())

# Create a class to play the game

In [ ]:
import gym

import random
from math import log10, floor
import numpy as np

def round_to(x, sig_figs):
    if x == 0.0:
        return 0.0
    if x < 0.0:
        return round(x, -int(floor(log10(abs(-x))) - (sig_figs - 1)))
    return round(x, -int(floor(log10(abs(x))) - (sig_figs - 1)))

class GamePlayer:

    def __init__(self):
        self.env = gym.make("CartPole-v0")
        self.qtable = {}
        
    def erase_training(self):
        self.qtable = {}
    
    def Q(self, state):
        if abs(state[0]) >= 2.2:
            x_pos = round(state[0], 2)
        else:
            x_pos = 0
        s = (x_pos, round(state[1], 1), round(state[2], 2), round_to(state[3], 1))
        if s not in self.qtable:
            self.qtable[s] = []
            for i in range(self.env.action_space.n):
                self.qtable[s].append(0.0)
        return self.qtable[s]
    
    def epison_q_action(self, state, epsilon):
        # 3. Choose an action a in the current world state (s)
        ## First we randomize a number
        exp_exp_tradeoff = random.uniform(0, 1)

        ## If this number > greater than epsilon --> exploitation (taking the biggest Q value for this state)
        if exp_exp_tradeoff > epsilon:
            action = np.argmax(self.Q(state))
        # Else doing a random choice --> exploration
        else:
            action = self.env.action_space.sample()
        return action
    
    def start_game(self, render = False):
        state = self.env.reset()
        if (render):
            self.env.render()
        return state

    def computer_play_step(self, state):
        action = np.argmax(self.Q(state))
        return self.play_game_step(action)
        
    def play_game_step(self, action, render = True):
        new_state, reward, done, info = self.env.step(action)
        if (render):
            self.env.render()
        return new_state, reward, done, info

    def end_game(self):
        self.env.close()
        
        
    def train(self, total_episodes, alpha, gamma, epsilon, decay_rate):
        self.start_game(False)
        # Exploration parameters
        max_epsilon = 0.8             # Exploration probability at start
        min_epsilon = 0.01            # Minimum exploration probability 
        alpha0 = alpha
        
        has_won = False
        reward_list = []
        tot_reward_list = []
        # 2 For life or until learning is stopped
        for episode in range(total_episodes):
            # Reset the environment
            state = self.env.reset()
            done = False
            tot_reward = 0
            while done is False:
                action = self.epison_q_action(state, epsilon)
                if action >= self.env.action_space.n:
                    raise IndexError
                # Take the action (a) and observe the outcome state(s') and reward (r)
                new_state, reward, done, info = self.play_game_step(action, False)

                # dirty ugly cheat witchcraft that does not even work
                # reward += abs(new_state[0])+10.0*abs(new_state[1])

                # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
                self.Q(state)[action] += alpha * (reward  + gamma * max(self.Q(new_state)) - self.Q(state)[action])

                # Our new state is state
                state = new_state
                tot_reward += reward
            reward_list.append(tot_reward)
            # epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay_rate*episode)
            
            if (episode+1) % 10000 == 0:
                ave_reward = np.mean(reward_list)
                tot_reward_list.append(ave_reward)
                reward_list = []
                #alpha = alpha0 * (total_episodes - episode)/total_episodes
                print('Episode {} Average Reward: {}, alpha: {}, e: {}, len(Q) {}'.format(episode+1, ave_reward, alpha, epsilon, len(self.qtable)))
        return tot_reward_list

# Training

In [ ]:
game = GamePlayer()

In [ ]:
total_episodes = 1000
alpha = 0.3
gamma = 0.9                 # Discounting rate
decay_rate = 5          # Exponential decay rate for exploration prob
epsilon = 0.3                 # Exploration rate
#game.erase_training()
rewards = game.train(total_episodes, alpha, gamma, epsilon, decay_rate)
print("Total reward average:", np.mean(rewards))
print(len(game.qtable))

In [ ]:
for episode in range(5):
    state = game.start_game()
    print("****************************************************")
    print("EPISODE ", episode)
    done = False
    tot_reward = 0
    while done is False:
    # for step in range(max_steps):
        # Take the action (index) that have the maximum expected future reward given that state
        new_state, reward, done, info = game.computer_play_step(state)
        #game.play_game_step(0)
        state = new_state
        tot_reward += reward
    print("Reward:", tot_reward)
game.end_game()

In [ ]:
game.qtable